# Learnings with Meachine Learnings Pipelines

# Titanic Without Using Pipeline

piplines chains together multiple steps so that the output of each step is used as input to the next step

pipelines makes it easy to apply the same preprocessing to train and test!

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
# from sklearn.transformer import ColumnTransformer
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import OrdinalEncoder

In [4]:
df= pd.read_csv('/kaggle/input/titanic/titanicdata.csv')

In [5]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
df.drop(columns= ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace= True)

In [8]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns= ['Survived']), 
                                                            df['Survived'],
                                                            test_size= 0.2, 
                                                            random_state= 42)

In [10]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [11]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [12]:
si_age= SimpleImputer()
si_embarked= SimpleImputer(strategy= 'most_frequent')

X_train_age= si_age.fit_transform(X_train[['Age']])
X_train_embarked= si_embarked.fit_transform(X_train[['Embarked']])

X_test_age= si_age.transform(X_test[['Age']])
X_test_embarked= si_embarked.transform(X_test[['Embarked']])

In [13]:
ohe_sex= OneHotEncoder(sparse= False, handle_unknown= 'ignore', dtype= int)
ohe_embarked= OneHotEncoder(sparse= False, handle_unknown= 'ignore', dtype= int)

X_train_sex= ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked= ohe_embarked.fit_transform(X_train_embarked)

X_test_sex= ohe_sex.transform(X_test[['Sex']])
X_test_embarked= ohe_embarked.transform(X_test_embarked)

In [14]:
# X_train_sex.shape

In [15]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [16]:
X_train_rem= X_train.drop(columns= ['Sex', 'Age', 'Embarked'])
X_test_rem= X_test.drop(columns= ['Sex', 'Age', 'Embarked'])

In [17]:
X_train_transformed= np.concatenate((X_train_rem, X_train_age, X_train_sex, X_train_embarked), axis= 1)
X_test_transformed= np.concatenate((X_test_rem, X_test_age, X_test_sex, X_test_embarked), axis= 1)

In [18]:
X_train_transformed.shape

(712, 10)

In [19]:
d_tree= DecisionTreeClassifier(random_state= 0)

d_tree.fit(X_train_transformed, y_train)

DecisionTreeClassifier(random_state=0)

In [20]:
y_pred= d_tree.predict(X_test_transformed)
y_pred

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0])

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.776536312849162

In [22]:
import pickle

In [23]:
pickle.dump(ohe_sex,open('/kaggle/working/ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('/kaggle/working/ohe_embarked.pkl', 'wb'))
pickle.dump(d_tree, open('/kaggle/working/d_tree.pkl', 'wb'))

print("Files saved in /kaggle/working/")

Files saved in /kaggle/working/


In [24]:
# Assume user input
# Pclass/gender/age/SibSp/Parch/Fare/Embarked
test_input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [25]:
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [26]:
test_input_sex= ohe_sex.transform(test_input[:, 1].reshape(1, 1))

In [27]:
test_input_sex

array([[0, 1]])

In [28]:
test_input_embarked= ohe_embarked.transform(test_input[:, -1].reshape(1,1))

In [29]:
test_input_embarked

array([[0, 0, 1]])

In [30]:
test_input_age = test_input[:,2].reshape(1,1)

In [31]:
test_input_age

array([[31.0]], dtype=object)

In [32]:
test_input_transformed = np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_embarked),axis=1)

In [33]:
test_input_transformed.shape

(1, 10)

In [34]:
d_tree.predict(test_input_transformed)

array([1])